In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
    
import openpyxl

from scipy import stats


In [3]:
df = pd.read_excel('附件_降维_XY.xlsx')

# 筛选Y染色体浓度大于等于0.04的行
filtered_df = df[df['Y染色体浓度'] >= 0.04]

# 按孕妇代码分组，并找到每组中检测孕周天数最小的行
result_df = filtered_df.loc[filtered_df.groupby('孕妇代码')['检测孕周_天数'].idxmin()]

# 选择需要的列
result_df = result_df[['序号', '孕妇代码', '孕妇BMI', 'Y染色体浓度', '检测孕周_天数']]

# 保存到新的Excel文件
result_df.to_excel('附件_提取.xlsx', index=False)


## 分位数进行bmi分组，对比

In [4]:
plt.rcParams['font.family'] = 'SimHei' 
sns.set(font='SimHei')
plt.rcParams['mathtext.fontset'] = 'stix'  # 使用STIX字体，支持数学符号
plt.rcParams['mathtext.default'] = 'regular'  # 默认数学文本样式

In [ ]:
def compare(data):
    """比较不同的分位数方法"""
    methods = {
        '等分位数': lambda x, n: np.percentile(x, [100 * i / n for i in range(1, n)]),
        '等间距': lambda x, n: np.linspace(np.min(x), np.max(x), n + 1)[1:-1],
        '基于密度的分位数': lambda x, n: np.percentile(x, [100 * i / (n+1) for i in range(1, n+1)])[:-1]
    }
    
    results = []
    bmi_values = data['孕妇BMI'].values
    gestational_age = data['检测孕周_天数'].values
    
    for method_name, method_func in methods.items():
        for n_groups in range(2, 7):
            try:
                boundaries = method_func(bmi_values, n_groups)
                labels = np.digitize(bmi_values, boundaries)
                
                total_variance = 0
                for i in range(n_groups):
                    group_ages = gestational_age[labels == i]
                    if len(group_ages) > 1:
                        total_variance += np.var(group_ages)
                
                results.append({
                    '方法': method_name,
                    '分组数': n_groups,
                    '方差和': total_variance,
                    '各组平均方差':total_variance / n_groups,
                    '边界': boundaries
                })
            except:
                continue
    
    return pd.DataFrame(results)

In [6]:
file_path = "D:\Mathematical-Contest-in-Modeling\C题\第二问\附件_提取.xlsx"
df = pd.read_excel(file_path, engine='openpyxl')

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\86185\AppData\Local\Temp\ipykernel_5780\559269292.py:1: SyntaxWarning: invalid escape sequence '\M'
  file_path = "D:\Mathematical-Contest-in-Modeling\C题\第二问\附件_提取.xlsx"


In [ ]:
comparison_results = compare(df)

In [8]:
# 导出比较结果到Excel
comparison_results_output_path = "D:\Mathematical-Contest-in-Modeling\C题\第二问\分位数方法比较结果.xlsx"

# 格式化边界列，使其更易读
comparison_results_formatted = comparison_results.copy()
comparison_results_formatted['边界'] = comparison_results_formatted['边界'].apply(
    lambda x: ', '.join([f'{b:.4f}' for b in x]) if isinstance(x, np.ndarray) else str(x)
)

# 保存到Excel
comparison_results_formatted.to_excel(comparison_results_output_path, index=False, engine='openpyxl')

print(f"分位数方法比较结果已保存到: {comparison_results_output_path}")
print("\n比较结果前5行:")
print(comparison_results_formatted.head())

分位数方法比较结果已保存到: D:\Mathematical-Contest-in-Modeling\C题\第二问\分位数方法比较结果.xlsx

比较结果前5行:
     方法  分组数          方差和      各组平均方差  \
0  等分位数    2   761.853550  380.926775   
1  等分位数    3  1132.358596  377.452865   
2  等分位数    4  1499.816099  374.954025   
3  等分位数    5  1849.588844  369.917769   
4  等分位数    6  2134.579644  355.763274   

                                            边界  
0                                      31.3046  
1                             30.2076, 32.7183  
2                    29.7521, 31.3046, 33.4103  
3           29.4376, 30.4834, 32.3113, 33.9906  
4  29.1803, 30.2076, 31.3046, 32.7183, 34.3329  


<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\86185\AppData\Local\Temp\ipykernel_5780\2870424238.py:2: SyntaxWarning: invalid escape sequence '\M'
  comparison_results_output_path = "D:\Mathematical-Contest-in-Modeling\C题\第二问\分位数方法比较结果.xlsx"


In [ ]:
# 1. 各组平均方差折线图
plt.figure(figsize=(10, 6))
methods = comparison_results['方法'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(methods)))

for i, method in enumerate(methods):
    method_data = comparison_results[comparison_results['方法'] == method]
    plt.plot(method_data['分组数'], method_data['各组平均方差'], 
             marker='o', label=method, color=colors[i], linewidth=2, markersize=8)
    
plt.xlabel('分组数量')
plt.ylabel('各组平均方差')
plt.title('不同分位数方法的各组平均方差比较')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('average_variance_comparison.svg', format='svg', bbox_inches='tight')
plt.show()

# 2. 各组平均方差热力图
plt.figure(figsize=(8, 6))
heatmap_data = []
for method in methods:
    method_data = comparison_results[comparison_results['方法'] == method]
    row = []
    for n_groups in range(2, 7):
        group_data = method_data[method_data['分组数'] == n_groups]
        if len(group_data) > 0:
            row.append(group_data['各组平均方差'].values[0])
        else:
            row.append(np.nan)
    heatmap_data.append(row)

heatmap = plt.imshow(heatmap_data, cmap='viridis_r', aspect='auto')
plt.colorbar(heatmap, label='各组平均方差')
plt.xticks(range(5), range(2, 7))
plt.yticks(range(len(methods)), methods)
plt.xlabel('分组数量')
plt.ylabel('分位数方法')
plt.title('各组平均方差热力图\n(颜色越浅表示方差越小)')

# 添加数值标注
for i in range(len(methods)):
    for j in range(5):
        if not np.isnan(heatmap_data[i][j]):
            plt.text(j, i, f'{heatmap_data[i][j]:.3f}', 
                    ha='center', va='center', fontweight='bold',
                    color='white' if heatmap_data[i][j] > np.nanmean(heatmap_data) else 'black')

plt.savefig('average_variance_heatmap.svg', format='svg', bbox_inches='tight')
plt.show()

# 3. 最佳分组结果散点图
plt.figure(figsize=(10, 6))
best_result = comparison_results.loc[comparison_results['各组平均方差'].idxmin()]
best_method = best_result['方法']
best_n_groups = best_result['分组数']
best_boundaries = best_result['边界']

# 应用最佳方法重新计算标签
if best_method == '等分位数':
    boundaries = np.percentile(df['孕妇BMI'], 
                              [100 * i / best_n_groups for i in range(1, best_n_groups)])
elif best_method == '等间距':
    boundaries = np.linspace(df['孕妇BMI'].min(), 
                           df['孕妇BMI'].max(), best_n_groups + 1)[1:-1]
else:  # 基于密度的分位数
    boundaries = np.percentile(df['孕妇BMI'], 
                              [100 * i / (best_n_groups+1) for i in range(1, best_n_groups+1)])[:-1]

labels = np.digitize(df['孕妇BMI'], boundaries)

# 绘制分组结果
colors = plt.cm.Set3(np.linspace(0, 1, best_n_groups))
for i in range(best_n_groups):
    group_data = df[labels == i]
    count = len(group_data)   
    mean_bmis = group_data['孕妇BMI'].mean() 
    plt.scatter(group_data['孕妇BMI'], group_data['检测孕周_天数'], 
               alpha=0.7, color=colors[i], label=f'组{i+1} (n={count}, 平均BMI={mean_bmis:.2f})', s=50)

for boundary in boundaries:
    plt.axvline(x=boundary, color='red', linestyle='--', alpha=0.8, linewidth=2)

plt.xlabel('孕妇BMI')
plt.ylabel('检测孕周天数')
plt.title(f'最佳方法: {best_method} ({best_n_groups}组)\n各组平均方差: {best_result["各组平均方差"]:.4f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('best_grouping_result.svg', format='svg', bbox_inches='tight')
plt.show()

# 输出结果
print("不同分位数方法的最佳分组结果（按各组平均方差排序）:")
print(comparison_results[['方法', '分组数', '方差和', '各组平均方差']].sort_values('各组平均方差'))

print(f"\n最佳分组方法: {best_method}, 分组数: {best_n_groups}")
print(f"各组平均方差: {best_result['各组平均方差']:.4f}")

In [14]:
bmi_values = df['孕妇BMI'].values
boundaries = np.linspace(bmi_values.min(), bmi_values.max(), best_n_groups + 1)[1:-1]

df2 = pd.read_excel("附件_降维_XY.xlsx")
bmi2_values = df2['孕妇BMI'].values
# 为每个样本分配组别
labels = np.digitize(bmi2_values, boundaries)
df2['组别'] = [int(i+1) for i in labels]

# 保存带有组别的新Excel文件
output_file_path = "D:\Mathematical-Contest-in-Modeling\C题\第二问\附件_分组_XY.xlsx"
df2.to_excel(output_file_path, index=False, engine='openpyxl')

print(f"\n已为原始数据添加组别列，并保存到: {output_file_path}")


# 显示前几行数据确认
print("\n添加组别后的数据前5行:")
print(df[['孕妇代码', '孕妇BMI', '检测孕周_天数', '组别']].head())

<>:11: SyntaxWarning: invalid escape sequence '\M'
<>:11: SyntaxWarning: invalid escape sequence '\M'
C:\Users\86185\AppData\Local\Temp\ipykernel_18524\2425527870.py:11: SyntaxWarning: invalid escape sequence '\M'
  output_file_path = "D:\Mathematical-Contest-in-Modeling\C题\第二问\附件_分组_XY.xlsx"



已为原始数据添加组别列，并保存到: D:\Mathematical-Contest-in-Modeling\C题\第二问\附件_分组_XY.xlsx

添加组别后的数据前5行:
   孕妇代码      孕妇BMI  检测孕周_天数  组别
0  A001  28.515625      141  组2
1  A002  33.331832       97  组3
2  A003  30.742188       91  组3
3  A004  28.641243       77  组2
4  A005  29.955556       86  组2
